In [1]:
import numpy as np

In [4]:
y_probs = np.random.rand(4,10,1)

In [7]:
y_probs[:,0,0]/np.sum(y_probs[:,0,0])

array([0.14888782, 0.26840907, 0.19936361, 0.3833395 ])

In [125]:
SymbolSet = ['a','b','c']
BeamWidth = 2

In [126]:
def InitializePaths(SymbolSet,y):
    PathScore = {}
    BlankPathScore = {}
    path = '-'
    BlankPathScore[path] = y[0]
    InitialPathWithFinalBlank = set({path})
    
    InitialPathWithFinalSymbol = set({})
    
    for c in SymbolSet:
        #path = c
        print(SymbolSet.index(c))
        PathScore[c] = y[SymbolSet.index(c)+1]
        InitialPathWithFinalSymbol.add(c) 
    
    return InitialPathWithFinalBlank,InitialPathWithFinalSymbol,BlankPathScore,PathScore
    #return InitialPathWithFinalBlank,InitialPathWithFinalSymbol
    

In [127]:
a,b,c,d = InitializePaths(SymbolSet,y_probs[:,0,0]/np.sum(y_probs[:,0,0]))

0
1
2


In [100]:
def Prune(PathsWithTerminalBlank,PathsWithTerminalSymbol,BlankPathScore,PathScore,BeamWidth):
    
    scorelist = []
    PrunedBlankPathScore = {}
    PrunedPathScore = {}
    
    for p in PathsWithTerminalBlank:
        scorelist.append(BlankPathScore[p])
        
    for p in PathsWithTerminalSymbol:
        scorelist.append(PathScore[p])
        
    cutoff = np.array(sorted(scorelist,reverse=True)[BeamWidth]) if BeamWidth < len(scorelist) else np.array(sorted(scorelist,reverse=True)[-1])
    print(cutoff)
    
    PrunedPathWithFinalBlank = set({})
    
    for p in PathsWithTerminalBlank:
        if BlankPathScore[p] > cutoff:
            PrunedPathWithFinalBlank.add(p)
            PrunedBlankPathScore[p] = BlankPathScore[p]
            
            
    
    PrunedPathWithFinalSymbol = set({})
    for p in PathsWithTerminalSymbol:
        if PathScore[p] > cutoff:
            PrunedPathWithFinalSymbol.add(p)
            PrunedPathScore[p] = PathScore[p]
            
    return  PrunedPathWithFinalBlank,PrunedPathWithFinalSymbol,PrunedBlankPathScore,PrunedPathScore

In [128]:
e,f,g,h = Prune(a,b,c,d,2)

0.1993636072822929


In [129]:
h

{'a': 0.26840907272720216, 'c': 0.3833394974987496}

In [27]:
0.21594328533091442* 0.3833394974987496

0.08277959048698184

In [132]:
def ExtendWithBlank(PathsWithTerminalBlank,PathsWithTerminalSymbol,BlankPathScore,PathScore,y):
    
    UpdatedPathsWithTerminalBlank = set({})
    UpdatedPathsWithTerminalBlank2 = set({})
    UpdatedBlankPathScore = {}
    UpdatedBlankPathScore2 = {}

    
    """if len(PathsWithTerminalBlank) == 0:
        UpdatedPathsWithTerminalBlank.add('-')
        UpdatedBlankPathScore['-'] = y[0]
        #print(UpdatedPathsWithTerminalBlank)"""
    
    #else:
        
    #for path in PathsWithTerminalBlank:
    #    UpdatedPathsWithTerminalBlank.add(path)
    #    UpdatedBlankPathScore[path] = BlankPathScore[path]*y[0]

    #print(UpdatedPathsWithTerminalBlank)

    for path in PathsWithTerminalBlank:
        #print(path)
        if '-' in path and len(path)>1:
            print('i m here')

            UpdatedPathsWithTerminalBlank.add(path.replace('-',''))

            UpdatedBlankPathScore[path.replace('-','')] = BlankPathScore[path]*y[0]
        else:
            UpdatedPathsWithTerminalBlank.add(path)
            UpdatedBlankPathScore[path] = BlankPathScore[path]*y[0]

    
    
    #if PathsWithTerminalSymbol!=0:
    for path in PathsWithTerminalSymbol:
        #if f ==1:
        if path in UpdatedPathsWithTerminalBlank:

            UpdatedBlankPathScore[path] = UpdatedBlankPathScore[path] + PathScore[path]*y[0]

        else:
            UpdatedPathsWithTerminalBlank.add(path)
            UpdatedBlankPathScore[path] = PathScore[path]*y[0]

    return  UpdatedPathsWithTerminalBlank,UpdatedBlankPathScore
             

In [106]:
y_probs[:,1,0]/np.sum(y_probs[:,1,0])

array([0.21594329, 0.20332033, 0.29778059, 0.2829558 ])

In [133]:
ExtendWithBlank(e,f,g,h,y_probs[:,1,0]/np.sum(y_probs[:,1,0]))

({'a', 'c'}, {'a': 0.057961136977336375, 'c': 0.08277959048698184})

In [110]:
ExtendWithBlank(e,f,g,h,y_probs[:,1,0]/np.sum(y_probs[:,1,0]))

{'-'}


({'a', 'c'}, {'a': 0.057961136977336375, 'c': 0.08277959048698184})

In [134]:
ExtendWithSymbol(e,f,g,h,y_probs[:,1,0]/np.sum(y_probs[:,1,0]),SymbolSet)

I m here


({'a', 'ab', 'ac', 'c', 'ca', 'cb'},
 {'a': 0.054573020582251615,
  'ab': 0.07992701127674202,
  'ac': 0.07594790389087216,
  'ca': 0.07794071219139195,
  'cb': 0.11415106064817468,
  'c': 0.10846813417220118})

In [135]:
def ExtendWithSymbol(PathsWithTerminalBlank,PathsWithTerminalSymbol,BlankPathScore,PathScore,y,SymbolSet):
    print('I m here')
    
    UpdatedPathsWithTerminalSymbol = set({})
    UpdatedPathScore = {}
    
    
    if len(PathsWithTerminalSymbol) != 0:
        for path in PathsWithTerminalSymbol:
            for c in SymbolSet:
                if c == path[-1]:
                    newpath = path
                else:
                    newpath = path + c

                UpdatedPathsWithTerminalSymbol.add(newpath)
                UpdatedPathScore[newpath] = PathScore[path]*y[SymbolSet.index(c)+1]
    else:
        for c in SymbolSet:
                newpath = c
                UpdatedPathsWithTerminalSymbol.add(newpath)
                UpdatedPathScore[newpath] = y[SymbolSet.index(c)+1]
        
    if len(PathsWithTerminalBlank) != 0:
        for path in PathsWithTerminalBlank:
             for c in SymbolSet:
                    newpath = path + c

                    if newpath in UpdatedPathsWithTerminalSymbol:
                        UpdatedPathScore[newpath] =  UpdatedPathScore[newpath] + BlankPathScore[path]*y[SymbolSet.index(c)+1]
                    else:
                        UpdatedPathsWithTerminalSymbol.add(newpath.replace('-',''))
                        UpdatedPathScore[newpath.replace('-','')] =  BlankPathScore[path]*y[SymbolSet.index(c)+1]
    return UpdatedPathsWithTerminalSymbol,UpdatedPathScore

In [136]:
a,b,c,d = InitializePaths(SymbolSet,y_probs[:,0,0]/np.sum(y_probs[:,0,0]))

0
1
2


In [137]:
print(a,b,c,d)

{'-'} {'a', 'b', 'c'} {'-': 0.14888782249175528} {'a': 0.26840907272720216, 'b': 0.1993636072822929, 'c': 0.3833394974987496}


In [138]:
for i in range(1,10):
    print(i)
    e,f,g,h = Prune(a,b,c,d,2)
    print(e,f,g,h)
    #print('Yo')
    a,c = ExtendWithBlank(e,f,g,h,y_probs[:,i,0]/np.sum(y_probs[:,i,0]))
    print(a,c)
    #print(i,j)
    b,d = ExtendWithSymbol(e,f,g,h,y_probs[:,i,0]/np.sum(y_probs[:,i,0]),SymbolSet)
    print(b,d)
    

1
0.1993636072822929
set() {'a', 'c'} {} {'a': 0.26840907272720216, 'c': 0.3833394974987496}
{'a', 'c'} {'a': 0.057961136977336375, 'c': 0.08277959048698184}
I m here
{'a', 'cb', 'ac', 'ca', 'c', 'ab'} {'a': 0.054573020582251615, 'ab': 0.07992701127674202, 'ac': 0.07594790389087216, 'ca': 0.07794071219139195, 'cb': 0.11415106064817468, 'c': 0.10846813417220118}
2
0.08277959048698184
set() {'cb', 'c'} {} {'cb': 0.11415106064817468, 'c': 0.10846813417220118}
{'cb', 'c'} {'cb': 0.03200204181668334, 'c': 0.030408843736064818}
I m here
{'cb', 'cba', 'cbc', 'ca', 'c'} {'cba': 0.038702878051353376, 'cb': 0.02274118078713748, 'cbc': 0.019513490633115017, 'ca': 0.0367760837743181, 'c': 0.018542025874680786}
3
0.03200204181668334
set() {'ca', 'cba'} {} {'cba': 0.038702878051353376, 'ca': 0.0367760837743181}
{'ca', 'cba'} {'ca': 0.0028977496611298096, 'cba': 0.003049570270893726}
I m here
{'cac', 'cba', 'ca', 'cbab', 'cab', 'cbac'} {'ca': 0.016174855425934127, 'cab': 0.0029609610250889823, 'cac':

In [61]:
d

{'-ba': 0.014376942704568156,
 '-b': 0.10042383080671154,
 '-bc': 0.030044853057330848,
 '-a': 0.011315921512715274,
 '-c': 0.023647948388205785}

In [69]:
x = 'a-bc-fr-'

In [72]:
x = x.replace('-','')

In [118]:
sorted_x = sorted(x.items(), key=lambda kv: kv[1])

In [122]:
sorted_x[-1][0]

'cbacbc'

In [117]:
{'cbacbc', 'cbacba', 'cbacbca', 'cbacbb', 'cbacbcb'} 
x ={'cbacbca': 2.6781941354896597e-05, 'cbacbcb': 0.0001960491688726125, 'cbacbc': 0.0004195754669868226, 'cbacba': 4.694782982137632e-05, 'cbacbb': 0.00034366750695505045}

In [ ]:
{'cbacbc': array([0.00195661]), 'cbacbcb': array([0.00061892]), 'cbacbb': array([0.00108495]), 'cbacbca': array([8.45496711e-05]), 'cbacba': array([0.00014821]), 'cbacb': array([0.00110791])}